<a href="https://colab.research.google.com/github/rahulgundala007/NLP_text_summarization/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch


In [ ]:
pip install accelerate -U

In [ ]:
from datasets import load_dataset

dataset = load_dataset('Rahulgundala007/ToSdataset2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer

model_name = 't5-small'  # You can choose other versions of T5 like 't5-base', 't5-large', etc.
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    # Prefix input text with 'summarize: '
    inputs = ["summarize: " + doc for doc in examples['ToS_Detail']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # No prefix for summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['ToS_Summary'], max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    per_device_train_batch_size=4,   # Batch size for training
    per_device_eval_batch_size=4,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    save_strategy="epoch",           # Save the model at the end of each epoch
    learning_rate=5e-5,              # Learning rate
    weight_decay=0.01,               # Weight decay for regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],   # Use your dataset splits appropriately
    eval_dataset=dataset['validation'] # Use your dataset splits appropriately
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.671835
2,No log,2.648273
3,No log,2.296098


TrainOutput(global_step=51, training_loss=4.88160915000766, metrics={'train_runtime': 542.4981, 'train_samples_per_second': 0.359, 'train_steps_per_second': 0.094, 'total_flos': 26391651287040.0, 'train_loss': 4.88160915000766, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./my_t5_summarizer')
tokenizer.save_pretrained('./my_t5_summarizer')


('./my_t5_summarizer/tokenizer_config.json',
 './my_t5_summarizer/special_tokens_map.json',
 './my_t5_summarizer/spiece.model',
 './my_t5_summarizer/added_tokens.json',
 './my_t5_summarizer/tokenizer.json')

**Epochs:** The training ran for 3 epochs. An epoch is a full pass through your entire training dataset.

**Training Loss**: This indicates how well the model is fitting the training data. Lower values mean better fit. "No log" suggests that the loss wasn't logged during the training but only at the end of each epoch for validation.

**Validation Loss**: Represents the model's performance on a separate part of the dataset not seen during training (the validation set). It's crucial for assessing how well the model generalizes to new, unseen data. Like training loss, lower is better. The validation loss decreased from 4.671835 to 2.296098, indicating that the model's predictions became more accurate with respect to the validation set as training progressed.

**global_step**: The total number of training steps completed. A step is a single gradient update; this number is 51, indicating your model's weights were updated 51 times.

**training_loss:** The average loss over all training steps. Here, it's somewhat higher than your final validation loss, which is typical as the model learns from the training data.

**metrics:** This dictionary contains various performance metrics:

**train_runtime:** Total training time in seconds (605.114).

**train_samples_per_second:** The number of training samples processed per second (0.322).

**train_steps_per_second**:The number of training steps (weight updates) performed per second (0.084).

**total_flos:** The total number of floating-point operations performed during training. This gives an idea of the computational cost (26,391,651,287,040 FLOPs).

**train_loss:** Repetition of the training loss.

**epoch**: The final epoch number (3.0).

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load the model and tokenizer
model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Prepare input text
input_text = "summarize: This is a long detailed text that you want to summarize. It may contain multiple sentences covering different aspects of a particular topic or subject matter."
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate the summary
output_tokens = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=150,
    num_beams=5,
    early_stopping=True
)

# Decode and print the summary
summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(summary)


this is a long detailed text that you want to summarize. it may contain multiple sentences covering different aspects of a particular topic or subject matter.


In [ ]:
pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    output_tokens = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=150,
        num_beams=5,
        early_stopping=True
    )
    summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return summary


In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer
import sacrebleu

# Load the model and tokenizer
model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Generate the summary
input_text = "summarize: This is a long detailed text that you want to summarize. It may contain multiple sentences covering different aspects of a particular topic or subject matter."
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
output_tokens = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_length=150,
    num_beams=5,
    early_stopping=True
)
summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Assume a reference summary for evaluation
reference_summary = "This text covers various aspects of a specific topic in detail."

# Compute BLEU score
bleu_score = sacrebleu.corpus_bleu([summary], [[reference_summary]])
print(f"Generated Summary: {summary}")
print(f"BLEU score: {bleu_score.score}")


Generated Summary: this is a long detailed text that you want to summarize. it may contain multiple sentences covering different aspects of a particular topic or subject matter.
BLEU score: 5.911245883122323


In [ ]:
pip install PyPDF2 transformers sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [17]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ''
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:  # Ensure there is text on the page
                text += page_text + ' '  # Append text of each page and a space
    return text


In [18]:
# Specify the path to your PDF file
pdf_path = '/content/apple-privacy-policy-en-ww.pdf'

# Call the function with the PDF path
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)


Apple Privacy Policy Apple’s Privacy Policy describes how Apple collects, uses, and shares your personal data. Updated December 22, 2022 
In addition to this Privacy Policy, we provide data and privacy information embedded in our products and certain features that ask to use your personal data. This product-specific information is accompanied by our Data & Privacy Icon.  You will be given an opportunity to review this product-specific information before using these features. You also can view this information at any time, either in Settings related to those features and/or online at apple.com/legal/privacy/data.  You can familiarize yourself with our privacy practices, accessible via the headings below, and contact us if you have questions. Apple Health Study Apps Privacy Policy 
 What Is Personal Data at Apple? At Apple, we believe strongly in fundamental privacy rights — and that those fundamental rights should not differ depending on where you live in the world. That’s why we treat 

In [19]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

model_path = './my_t5_summarizer'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_summary(text):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    output_tokens = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=150,  # Adjust based on expected summary length
        num_beams=5,
        early_stopping=True
    )
    summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return summary


In [20]:
import sacrebleu

def compute_bleu(generated_summary, reference_summary):
    bleu_score = sacrebleu.corpus_bleu([generated_summary], [[reference_summary]])
    return bleu_score.score


In [21]:
pdf_path = '/content/apple-privacy-policy-en-ww.pdf'
pdf_text = extract_text_from_pdf(pdf_path)
generated_summary = generate_summary(pdf_text)
print("Generated Summary:", generated_summary)

# You must provide this
reference_summary = "Your reference summary here."
bleu_score = compute_bleu(generated_summary, reference_summary)
print(f"BLEU score: {bleu_score}")


Generated Summary: Apple’s Privacy Policy describes how Apple collects, uses, and shares your personal data. this information is accompanied by our Data & Privacy Icon. you will be given an opportunity to review this product-specific information before using these features.
BLEU score: 0.8522452865706931
